<a href="https://colab.research.google.com/github/Kryvkodenis/Journey-to-Springfield-Kaggle/blob/master/ResNet50_simpsons_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler

import numpy as np
import pandas as pd
from scipy.special import softmax
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path


In [ ]:
def seed_torch(seed=1029):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

seed_torch()

In [ ]:
class Simpsons(Dataset):
    
    def __init__(self, files, mode):
        super().__init__()
        self.files = sorted(files)
        self.mode = mode

        if self.mode not in DATA_MODES:
            print(f'{self.mode} is not corret; correct modes: {DATA_MODES}')
            raise NameError

        self.len_ = len(self.files)

        self.label_encoder = LabelEncoder()

        if self.mode != 'test':
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)

            with open ('Label_encoder.pkl', "wb") as le_dump_file:
                pickle.dump(self.label_encoder, le_dump_file)

    def __len__(self):
        return self.len_

    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image

    def __getitem__(self, index):
        np.random.seed(12)
        transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])])
        
        transform_for_train = transforms.Compose([transforms.RandomCrop(200),
                                                transforms.RandomRotation(10),
                                                transforms.RandomGrayscale(0.4)
                                                ])
        x = self.load_sample(self.files[index])
        if self.mode == 'train':
            x = transform_for_train(x)
        x = self._prepare_sample(x)
        x = np.array(x / 255, dtype='float32')
        x = transform(x)
        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y

    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        return np.array(image)


In [ ]:
def train(train_files, sampler_train_wht, val_files, model, epochs, batch_size):
    train_loader = DataLoader(train_files, batch_size=batch_size, sampler=sampler_train_wht, num_workers=4)
    val_loader = DataLoader(val_files, batch_size=batch_size, shuffle=False, num_workers=4) #val_dataset
    
    best_acc = 0
    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
                    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    opt = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
    #opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'min', 0.1, 3, verbose=True)
    criterion = nn.CrossEntropyLoss()

    for epoch in tqdm_notebook(range(epochs)):
        train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt, batch_size)
        val_loss, val_acc = eval_epoch(model, val_loader, criterion)
        scheduler.step(train_loss)
        history. append((train_loss, train_acc, val_loss, val_acc))
        if val_acc > best_acc:
            
            state = {'epoch': epoch,
                    'state_dict': model.state_dict(),
                    'optimizer': opt.state_dict()
                    }
            try:
                torch.save(state, '/content/drive/My Drive/simpsons/models/ckpt_d{}.pth'.format('fineTuned'))
            except OSError:
                continue
            best_acc = val_acc

        print(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                    v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))

    return history

In [ ]:
def fit_epoch(model, train_loader, criterion, optimizer, batch_size):
    running_loss= 0.0
    running_corrects = 0
    processed_data = 0
    
    with tqdm_notebook(desc="epoch", total=245) as pbar_outer1:
        for inputs, labels in train_loader:
            inputs = inputs + torch.empty(*inputs.size()).normal_(0, 0.0001)
            
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)
            optimizer.zero_grad()

            outputs = model(inputs)
            #print(list(outputs.size()))
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            preds = torch.argmax(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            processed_data += inputs.size(0)

            pbar_outer1.update(1)

    train_loss = running_loss / processed_data
    train_acc = running_corrects.cpu().numpy() / processed_data
    return train_loss, train_acc

In [ ]:
def eval_epoch(model, val_loader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    processed_size = 0
    
    for inputs, labels in val_loader:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        with torch.set_grad_enabled(False):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            preds = torch.argmax(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        processed_size += inputs.size(0)
    val_loss = running_loss / processed_size
    val_acc = running_corrects.double() / processed_size
    
    return val_loss, val_acc

In [ ]:
def predict(model, test_loader):
    with torch.no_grad():
        logits = []
    
        for inputs in test_loader:
            inputs = inputs.to(DEVICE)
            model.eval()
            outputs = model(inputs).cpu()
            logits.append(outputs)
            
    probs = nn.functional.softmax(torch.cat(logits), dim=-1).numpy()
    return probs

In [ ]:
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE = 224
# работаем на видеокарте
DEVICE = torch.device("cuda")

In [ ]:
TRAIN_DIR = Path('/content/drive/My Drive/simpsons/train/simpsons_dataset')
TEST_DIR = Path('/content/drive/My Drive/simpsons/testset/testset')

train_val_files = sorted(list(TRAIN_DIR.rglob('*.jpg')))
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))

train_val_labels = [path.parent.name for path in train_val_files]
train_files, val_files = train_test_split(train_val_files, test_size=0.25, \
                                          stratify=train_val_labels)

n_classes = len(np.unique(train_val_labels))

val_dataset = Simpsons(val_files, mode='val')
    
train_dataset = Simpsons(train_files, mode='train')

In [ ]:
simpson_np = np.array(train_val_labels)
dct_simpsons = pd.Series(simpson_np).value_counts().to_dict()

In [ ]:
dct_simpsons_wht = {}
for key in dct_simpsons:
    dct_simpsons_wht.update({key:1. / dct_simpsons[key]})

In [ ]:
train_labels = [path.parent.name for path in sorted(train_files)]
weights = [dct_simpsons_wht[x] for x in train_labels]

In [ ]:
sampler_train_wht = torch.utils.data.WeightedRandomSampler(weights, len(weights))

ResNet 50

In [ ]:
ResNet50 = models.resnet50(pretrained=True)


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


2 loops of training. (1 train loop : train classifier, then train last bottleneck.)

In [ ]:
num_features = 2048
ResNet50.fc =nn.Linear(num_features,n_classes)
ResNet50 = ResNet50.to(DEVICE)

In [ ]:
history = train(train_dataset, sampler_train_wht, val_dataset, model=ResNet50, epochs=50, batch_size=64)



Epoch 001 train_loss: 0.5161                     val_loss 0.0751 train_acc 0.8660 val_acc 0.9822




Epoch 002 train_loss: 0.4985                     val_loss 0.0854 train_acc 0.8693 val_acc 0.9784




Epoch 003 train_loss: 0.4823                     val_loss 0.0745 train_acc 0.8737 val_acc 0.9816




Epoch 004 train_loss: 0.4778                     val_loss 0.0743 train_acc 0.8763 val_acc 0.9835




Epoch 005 train_loss: 0.4700                     val_loss 0.0861 train_acc 0.8779 val_acc 0.9793


...



Epoch 015 train_loss: 0.3631                     val_loss 0.0829 train_acc 0.9024 val_acc 0.9816




Epoch 016 train_loss: 0.3458                     val_loss 0.0786 train_acc 0.9091 val_acc 0.9797




Epoch 017 train_loss: 0.3334                     val_loss 0.0868 train_acc 0.9118 val_acc 0.9780




Epoch 018 train_loss: 0.3390                     val_loss 0.0818 train_acc 0.9098 val_acc 0.9807




Epoch 019 train_loss: 0.3322                     val_loss 0.0970 train_acc 0.9126 val_acc 0.9753


In [ ]:
ResNet50_chekpoint = torch.load('/content/drive/My Drive/simpsons/models/ckpt_{}.pth'.format('fineTuned'))
ResNet50.load_state_dict(ResNet50_chekpoint['state_dict'])

<All keys matched successfully>

In [ ]:
for param in ResNet50.fc.parameters():
    param.requires_grad = True


In [ ]:
history = train(train_dataset, sampler_train_wht, val_dataset, model=ResNet50, epochs=50, batch_size=64)

In [ ]:
ResNet50_chekpoint = torch.load('/content/drive/My Drive/simpsons/models/ckpt_{}.pth'.format('fineTuned'))
ResNet50.load_state_dict(ResNet50_chekpoint['state_dict'])

<All keys matched successfully>

In [ ]:
for param in ResNet50.layer3.parameters():
    param.requires_grad = True

In [ ]:
for param in ResNet50.fc.parameters():
    param.requires_grad = True

In [ ]:
history = train(train_dataset, sampler_train_wht, val_dataset, model=ResNet50, epochs=50, batch_size=64)

In [ ]:
ResNet50_chekpoint = torch.load('/content/drive/My Drive/simpsons/models/ckpt_{}.pth'.format('fineTuned'))
ResNet50.load_state_dict(ResNet50_chekpoint['state_dict'])

To submit

In [ ]:
idxs = list(range(0, 1000))
imgs = [val_dataset[id][0].unsqueeze(0) for id in idxs]
probs_ims = predict(ResNet50, imgs)

In [ ]:
label_encoder = pickle.load(open("Label_encoder.pkl", 'rb'))
y_pred = np.argmax(probs_ims,-1)

actual_labels = [val_dataset[id][1] for id in idxs]

preds_class = [label_encoder.classes_[i] for i in y_pred]

In [ ]:
from sklearn.metrics import f1_score

f1_score(actual_labels, y_pred, average='weighted',sample_weight= weights[:1000])

In [ ]:

test_dataset = Simpsons(test_files, mode="test")
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=64, num_workers=4)
probs = predict(ResNet50, test_loader)


preds = label_encoder.inverse_transform(np.argmax(probs, axis=1))
test_filenames = [path.name for path in test_dataset.files]

In [ ]:
my_submit = pd.DataFrame({'Id': test_filenames, 'Expected': preds})
my_submit.to_csv('ResNet50_submit.csv', index=False)

In [ ]:
preds_class = np.array(preds_class)
Error_pred = pd.Series(preds_class[y_pred != actual_labels]).value_counts()

In [ ]:
wrong_classes = list(Error_pred.axes[0])
wrong_classes

In [ ]:
train_val_for_wrong_clases = [x for x in train_val_files if x.parent.name in wrong_classes] 

In [ ]:
wrong_clases_labels = [path.parent.name for path in train_val_for_wrong_clases]
train_wrong_clases_files, val_wrong_clases_files = train_test_split(train_val_for_wrong_clases, test_size=0.25, \
                                          stratify=wrong_clases_labels )

n_wrong_classes = len(np.unique(wrong_clases_labels))

val_wrong_clases_dataset = Simpsons(val_wrong_clases_files, mode='val')
    
train_wrong_clases_dataset = Simpsons(train_wrong_clases_files, mode='train')

In [ ]:
dct_wrong_classes = pd.Series(wrong_clases_labels).value_counts().to_dict()
dct_wrong_classes_wht = {}
for key in dct_wrong_classes:
    dct_wrong_classes_wht.update({key:1. / dct_wrong_classes[key]})

train_wrong_classes_labels = [path.parent.name for path in sorted(train_wrong_clases_files)]
weights_wrong_classes = [dct_wrong_classes_wht[x] for x in train_wrong_classes_labels]

In [ ]:
sampler_train_wrong_classes = torch.utils.data.WeightedRandomSampler(weights_wrong_classes, len(weights_wrong_classes))

In [ ]:
def train_2(train_files, val_files, model, epochs, batch_size):
    train_loader = DataLoader(train_files, shuffle=True, batch_size=batch_size, num_workers=4)
    val_loader = DataLoader(val_files, batch_size=batch_size, shuffle=False, num_workers=4) #val_dataset
    
    best_acc = 0
    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
                    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    opt = torch.optim.SGD(model.parameters(), lr=1e-5, momentum=0.9)
    #opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'max', 0.1, 3, verbose=True)
    criterion = nn.CrossEntropyLoss()

    for epoch in tqdm_notebook(range(epochs)):
        train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt, batch_size)
        val_loss, val_acc = eval_epoch(model, val_loader, criterion)
        scheduler.step(val_acc)
        history. append((train_loss, train_acc, val_loss, val_acc))
        if val_acc > best_acc:
            
            state = {'epoch': epoch,
                    'state_dict': model.state_dict(),
                    'optimizer': opt.state_dict()
                    }
            try:
                torch.save(state, '/content/drive/My Drive/simpsons/models/ckpt_Res_2.pth')
            except OSError:
                continue
            best_acc = val_acc

        print(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                    v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))

    return history

In [ ]:
ResNet50_2 = models.resnet50(pretrained=True)

In [ ]:
num_features = 2048
ResNet50_2.fc =nn.Linear(num_features, 7)
#ResNet50_2 = ResNet50_2.to(DEVICE)

In [ ]:
ResNet50_2_chekpoint = torch.load('/content/drive/My Drive/simpsons/models/ckpt_Res_2.pth')
ResNet50_2.load_state_dict(ResNet50_2_chekpoint['state_dict'])

<All keys matched successfully>

In [ ]:
for param in ResNet50_2.fc.parameters():
    param.requires_grad = True

In [ ]:
train_2(train_wrong_clases_dataset, val_wrong_clases_dataset, ResNet50_2, 25, 64)

In [ ]:
class concNets(nn.Module):
  def __init__(self, n_class):
    super().__init__()
    self.Resnet_1 = ResNet50
    self.Resnet_2 = ResNet50_2
    self.fc_p = nn.Sequential(nn.Linear(42+7, 42+7),
                              nn.ReLU())
    self.fc = nn.Linear(42 + 7, n_class)

  def forward(self, x):
    out_1 = self.Resnet_1(x)
    out_2 = self.Resnet_2(x)
    x = torch.cat((out_1, out_2), dim=1)
    x = self.fc_p(x)
    x = self.fc(x)
    return x


In [ ]:
m = concNets(42)

In [ ]:
m.fc

Linear(in_features=49, out_features=42, bias=True)

In [ ]:
import torchsummary
torchsummary.summary(m.cuda(), (3, 244, 244))

In [ ]:
def train_concat(train_files, val_files, model, epochs, batch_size):
    train_loader = DataLoader(train_files, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_files, batch_size=batch_size, shuffle=False, num_workers=4) #val_dataset
    
    best_acc = 0
    history = []
    log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
                    val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

    opt = torch.optim.SGD(model.fc.parameters(), lr=1e-4, momentum=0.9)
    #opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, 'max', 0.1, 3, verbose=True)
    criterion = nn.CrossEntropyLoss()

    for epoch in tqdm_notebook(range(epochs)):
        train_loss, train_acc = fit_epoch(model, train_loader, criterion, opt, batch_size)
        val_loss, val_acc = eval_epoch(model, val_loader, criterion)
        scheduler.step(val_acc)
        history. append((train_loss, train_acc, val_loss, val_acc))
        if val_acc > best_acc:
            
            state = {'epoch': epoch,
                    'state_dict': model.state_dict(),
                    'optimizer': opt.state_dict()
                    }
            try:
                torch.save(state, '/content/drive/My Drive/simpsons/models/ckpt_concat_model{}.pth'.format('fineTuned'))
            except OSError:
                continue
            best_acc = val_acc

        print(log_template.format(ep=epoch+1, t_loss=train_loss,\
                                    v_loss=val_loss, t_acc=train_acc, v_acc=val_acc))

    return history

In [ ]:
for param in m.Resnet_2.parameters():
    param.requires_grad = False
for param in m.Resnet_1.parameters():
    param.requires_grad = False

In [ ]:
m = m.to(DEVICE)

In [ ]:
history = train_concat(train_dataset, val_dataset, model=m, epochs=50, batch_size=64)